# Imports

In [24]:
import os
import json
from typing import List

# Lectura json

In [7]:
data_path = os.path.join(os.path.dirname(os.getcwd()), 'Data')

In [10]:
jsons = [ x for x in os.listdir(data_path) if  x.endswith('.json')]
jsons

['2024-I+D_Inicitiavas finalizadas_slides cierre con links (1).json']

In [14]:
# A partir de los jsons enlistados arriba elegir el que corresponda al último archivo de iniciativas finalizadas
# Si solo hay uno, solo ejecutar la celda
data_json = os.path.join(data_path, jsons[0])
data_json

'c:\\Users\\fmromero\\Documents\\Repositorios\\Variadas-Registro-Ini-Cerradas\\Data\\2024-I+D_Inicitiavas finalizadas_slides cierre con links (1).json'

In [18]:
with open(data_json, 'r', encoding='utf-8') as data_to_read:
    data = json.load(data_to_read)
print(data)

[{'slide_number': 1, 'texts': ['EXPERTOS\xa0EN RESOLVER PROBLEMAS QUE IMPACTAN AL NEGOCIO A TRAVÉS DE LA TECNOLOGÍA', 'SEGUIMIENTO INICIATIVAS I+D 2024']}, {'slide_number': 2, 'texts': ['Practia I+D lleva adelante un conjunto de iniciativas de investigación que son estratégicas para el posicionamiento de la empresa.', 'TEMARIO Avances de Equipos y Proyectos']}, {'slide_number': 3, 'texts': ['Equipo I+D', 'Copyright © 2024 Practia, a company of Publicis Sapient . All rights reserved .', '1)  LINK ACCESO A PRESENTACION Presentación Iniciativas.pptx (link: https://publicisgroupe.sharepoint.com/:p:/r/sites/ID-Equipo/Shared%20Documents/General/Seguimiento%20Iniciativas%20I+D%20_2024/INICIATIVAS%20CERRADAS_%20Grabaciones-Demo-PPT/01_Asistentes_CONVERSACIONALES/COMPARATIVO%20Asistentes/Presentaci%C3%B3n%20Iniciativas.pptx?d=wb07a8304961c4c4db7c687321e7ba2f9&csf=1&web=1&e=dhjcqq)', '3) LINK ACCESO A CODIGO https://dev.azure.com/AR-Practia-ID/Asist-Pol-Int (link: https://dev.azure.com/AR-Practi

In [22]:
# WARNING: Tomo como precondición que las dos últimas diapositivas y las dos primeras son innecesarias.
slides = data[2:-2]
slides

[{'slide_number': 3,
  'texts': ['Equipo I+D',
   'Copyright © 2024 Practia, a company of Publicis Sapient . All rights reserved .',
   '1)  LINK ACCESO A PRESENTACION Presentación Iniciativas.pptx (link: https://publicisgroupe.sharepoint.com/:p:/r/sites/ID-Equipo/Shared%20Documents/General/Seguimiento%20Iniciativas%20I+D%20_2024/INICIATIVAS%20CERRADAS_%20Grabaciones-Demo-PPT/01_Asistentes_CONVERSACIONALES/COMPARATIVO%20Asistentes/Presentaci%C3%B3n%20Iniciativas.pptx?d=wb07a8304961c4c4db7c687321e7ba2f9&csf=1&web=1&e=dhjcqq)',
   '3) LINK ACCESO A CODIGO https://dev.azure.com/AR-Practia-ID/Asist-Pol-Int (link: https://dev.azure.com/AR-Practia-ID/Asist-Pol-Int) (Conversacional) AI-Gen-Contratos-Legales - Repos (azure.com) (link: https://dev.azure.com/AR-Practia-ID/_git/AI-Gen-Contratos-Legales?path=%2F&version=GBprod&_a=contents) (No Conversacional)',
   '2) LINK ACCESO A DEMO Cierre Iniciativas I+D-20240809_103534-Grabación de la reunión.mp4 (link: https://publicisgroupe-my.sharepoint.c

# Funciones

In [136]:
def name_of(data: List[str]) -> str:
    """
    Obtiene el nombre del proyecto a partir de una lista de textos extraídos de la diapositiva.

    Precondición:
      - Existe un elemento con formato proyecto: <nombre> (puede o no haber espacios o mayusculas)
    
    params:
      - data: Una lista con las cadenas de texto extraidas de la ppt.
    
    returns:
      Optional(str): El nombre del proyecto o None si no se cumplió la precondición.
    """
    for elem in data:
        split_elem = elem.split(':')
        if split_elem[0].lower().startswith('proyecto') and len(split_elem) > 1:
            return split_elem[1].replace('\xa0', '')

In [122]:
def is_seniority(word: str) -> bool:
    """
    Indica si la palabra dada es parte de un seniority o similar.

    params:
      - word: La palabra a ser evaluada.
    
    returns:
      bool: True si la palabra es una seniority.
    
    Observación: La lista de palabras consideradas seniority es arbitraria y puede requerir que se agreguen nuevas.
    Toda palabra agregada debe serlo en minuscula.
    """
    return word.lower().strip() in [
          'fs','unqui', 'unsam', 'de', 'ds', 'dev', 'developer', 'data', 'ssr',
          'cientifico', 'ingeniera', 'datos', 'scientist', 'famaf', 'desarrollador',
          'téc', 'en', 'fullstack', 'jr', 'nombre', 'perfil', 'tec', 'programación',
          'programacion', 'universidad', 'unq', 'quilmes', 'informática', 'técnico', 'tpi', 'lic',
          'técnica', 'engineer', ' ', 'Técnico', ' '
        ]

In [139]:
def flat_names_filtering_seniorities(names: List[str]) -> List[str]:
    """
    Obtiene una lista de nombres a partir de una lista de palabras.

    precondición:
      - Es necesario que los nombres esten continuados y separados por sus correspondientes seniorities.
           por ej: ['Juan', 'Lorenzo', 'Senior', 'Miguel', 'Perez', 'jr']
                   donde Juan Lorenzo es Senior y Miguel Perez es jr.
    params:
      - names: Una lista de palabras que tienen nombres y seniorities.
    
    returns:
      list(str): Una lista de nombres y apellidos, donde cada elemento es el nombre y apellido de una persona.
          por ej: ['Juan Lorenzo', 'Miguel Perez']
    """
    flatten = []
    i = 0
    new_name = True
    for word in names:
        if new_name and not is_seniority(word):
            new_name = False
            flatten.append(word.strip())
            i = len(flatten) - 1
            continue
        
        if not new_name and not is_seniority(word):
            flatten[i] = flatten[i] + ' ' + word.strip()
            continue
        
        if not new_name and is_seniority(word):
            new_name = True
    return flatten

In [128]:
def remove_characters(lst: List[str], characters: List[str]) -> List[str]:
    """
    Dada una lista de palabras y una lista de caracteres, reemplaza dichos caracteres en la lista de 
    palabras por espacios en blanco.

    params:
      - lst: La lista de palabras con caracteres a reemplazar.
      - characters: La lista de caracteres a reemplazar.

    returns:
      list(str): La lista lst pero con espacios donde antes estaban los caracteres dados. 
    """
    for char in characters:
        lst = [s.replace(char, ' ') for s in lst]
    return lst

In [129]:
def members_of(data: List[str]) -> List[str]:
    """
    Obtiene el nombre de los miembros del proyecto a partir de una lista de textos extraídos de la diapositiva.

    Precondición:
      - Existe un elemento con formato proyecto: <nombre> <seniority> <nombre> <seniority> (puede o no haber mayusculas)
    
    params:
      - data: Una lista con las cadenas de texto extraidas de la ppt.
    
    returns:
      List(str): La lista de nombres de los miembros del proyecto.
    """
    for elem in data:
        if elem.lower().startswith('equipo') and not elem == 'Equipo I+D':
            names = elem.split()[1:]
            names = [s.replace('\xa0', ' ') for s in names]
            names = remove_characters(names, ['\\', '/', '.', '(', ')', '-', ',', ' '])
            lst_names = flat_names_filtering_seniorities(names)
            return lst_names

# Obtención proyectos

In [155]:
projects = []
for slide in slides:
    texts = slide['texts']
    members = members_of(texts)
    project_name = name_of(texts)
    print("----------------------------------------------------------------------------------------------------------------")
    #print(texts)
    print(f"En el proyecto: \n{project_name}\nLos miembros son: {members}")
    tuples = [(name.strip(), project_name.strip()) for name in members]
    projects.extend(tuples)

----------------------------------------------------------------------------------------------------------------
En el proyecto: 
 Comparativo de asistentes
Los miembros son: ['Pablo Bilocopetiuc ', 'Leandro Amarilla ']
----------------------------------------------------------------------------------------------------------------
En el proyecto: 
 IMPLEMENTACIÓN Whatsapp
Los miembros son: ['Pablo Bilocopetiuc ', 'Leandro Amarilla ', 'Rodrigo Alonso ']
----------------------------------------------------------------------------------------------------------------
En el proyecto: 
 Segmentación clientes – Fabric
Los miembros son: ['Lucas Rodrigez ', 'Natalia Mazzeo']
----------------------------------------------------------------------------------------------------------------
En el proyecto: 
 Segmentación clientes – AZURE ML
Los miembros son: ['Luis Ibañez ', 'Javier Lassa ', 'Javier Olmos ']
--------------------------------------------------------------------------------------------

In [156]:
# Remuevo elementos sin nombre de miembro
projects = [x for x in projects if len(x[0]) != 0]
projects

[('Pablo Bilocopetiuc', 'Comparativo de asistentes'),
 ('Leandro Amarilla', 'Comparativo de asistentes'),
 ('Pablo Bilocopetiuc', 'IMPLEMENTACIÓN Whatsapp'),
 ('Leandro Amarilla', 'IMPLEMENTACIÓN Whatsapp'),
 ('Rodrigo Alonso', 'IMPLEMENTACIÓN Whatsapp'),
 ('Lucas Rodrigez', 'Segmentación clientes – Fabric'),
 ('Natalia Mazzeo', 'Segmentación clientes – Fabric'),
 ('Luis Ibañez', 'Segmentación clientes – AZURE ML'),
 ('Javier Lassa', 'Segmentación clientes – AZURE ML'),
 ('Javier Olmos', 'Segmentación clientes – AZURE ML'),
 ('Lucas Rodrigez', 'Segmentación clientes – Fabric'),
 ('Natalia Mazzeo', 'Segmentación clientes – Fabric'),
 ('Luis Ibañez', 'Segmentación clientes – AZURE ML'),
 ('Javier Lassa', 'Segmentación clientes – AZURE ML'),
 ('Javier Olmos', 'Segmentación clientes – AZURE ML'),
 ('Lucas Rodrigez', 'FORECAST -FABRIC'),
 ('Natalia Mazzeo', 'FORECAST -FABRIC'),
 ('Alexis Pacek', 'Forecast azure ML'),
 ('Carlos Jurado', 'Forecast azure ML'),
 ('Luis Ibañez', 'Forecast azure 

In [157]:
set([x[0] for x in projects])

{'Adrian David Cardozo',
 'Alexis Pacek',
 'Antonieta Kuz',
 'Belén Allegue',
 'Belén Allegue Data',
 'Camila Pía Scaglioni Ochalde',
 'Carlos Jurado',
 'Darian Yane',
 'Dayana Monsalve',
 'Denise Martin',
 'Ezequiel Oyola TPI Florencia Rossi UNSAM',
 'Federico Guilenea',
 'Fernando Mario Romero Muñoz',
 'Fernando Pereira',
 'Florencia Rossi',
 'Full Stack Ezequiel Oyola Técnico',
 'Garcia Juan Ignacio',
 'Gimena Del Valle Ibañez',
 'Hugo Isaias Chico',
 'Javier Lassa',
 'Javier Olmos',
 'Joaquin Ignacio Brito',
 'Joaquín Feltes',
 'Josefina Volosin',
 'Juan Augusto Vaca',
 'Juan Cruz Insaurralde',
 'Laura Montes',
 'Leandro Amarilla',
 'Leandro Nehuen Amarilla',
 'Loana Schleich',
 'Loana Schleich García',
 'Lopez Mariano',
 'Lucas Rodrigez',
 'Lucía Pereyra',
 'Luis Fernando Ibanez Mario Javier Olmos',
 'Luis Fernando Ibañez Gimena del Valle Ibañez Javier Olmos Lucas Exequiel Rodriguez',
 'Luis Ibañez',
 'Maria Rosa Morales Gimena del valle Ibañez Javier Nahuel Lassa Ortiz',
 'Marian

In [158]:
set([x[1] for x in projects])

{'',
 'AI Testing',
 'Anomalias -FABRIC',
 'Anomalias azure ML',
 'Asistente Politicas Internas',
 'CONTRATOS LEGALES',
 'Comparativo de asistentes',
 'Conversacional Audio formularios',
 'Detección de indumentaria',
 'FORECAST -FABRIC',
 'Forecast azure ML',
 'Generador Ideas comerciales',
 'HUMAN FEEDBACK',
 'IMPLEMENTACIÓN Whatsapp',
 'LLM Research',
 'Lubricantes enex',
 'MINERA CENTINELA',
 'Minera centinela – RL – ETAPA 1',
 'Segmentación clientes – AZURE ML',
 'Segmentación clientes – Fabric',
 'Testeo de modelos de IA generativa',
 'Video indexer – etapa 1',
 'YPF - FLUJOGRAMAS',
 'stonex'}